# Note
* 1.loading data
* 2.tokenizing data to the list
* 3.build the vacabulary to integer
* 4.padding the doc

In [1]:
import locale
import glob
import os.path
import requests
import tarfile
import sys
import codecs
from smart_open import smart_open
import re
import nltk
import numpy as np
import pickle
import itertools
from collections import Counter, OrderedDict
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from keras.preprocessing.sequence import pad_sequences
import re
from os import listdir
import string
from nltk.corpus import stopwords
from util.util_functions import getWordIdx, load_embedding_matrix_gensim
import gensim
import pickle
from tensorflow import keras
import tensorflow as tf
from keras.layers import Conv1D,Embedding,MaxPooling1D,Input
from keras.models import Model
import pandas as pd

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
dirname = 'data/aclImdb'
filename = 'aclImdb_v1.tar.gz'
locale.setlocale(locale.LC_ALL, 'C')
all_lines = []

if sys.version > '3':
    control_chars = [chr(0x85)]
else:
    control_chars = [unichr(0x85)]

# Convert text to lower-case and strip punctuation/symbols from words
def normalize_text(text):
#     norm_text = text.lower()
    # Replace breaks with spaces
    norm_text = text.replace('<br />', ' ')
    # Pad punctuation with spaces on both sides
    norm_text = re.sub(r"([\[\].\",()!?;:/])", " \\1 ", norm_text)
    return norm_text

if not os.path.isfile('data/aclImdb/alldata-id.txt'):
#     if not os.path.isdir(dirname):
#         if not os.path.isfile(filename):
#             # Download IMDB archive
#             print("Downloading IMDB archive...")
#             url = u'http://ai.stanford.edu/~amaas/data/sentiment/' + filename
#             r = requests.get(url)
#             with smart_open(filename, 'wb') as f:
#                 f.write(r.content)
#         # if error here, try `tar xfz aclImdb_v1.tar.gz` outside notebook, then re-run this cell
#         tar = tarfile.open(filename, mode='r')
#         tar.extractall()
#         tar.close()
#     else:
#         print("IMDB archive directory already available without download.")

    # Collect & normalize test/train data
    print("Cleaning up dataset...")
    folders = ['train/pos', 'train/neg', 'test/pos', 'test/neg']
    for fol in folders:
        temp = u''
        newline = "\n".encode("utf-8")
        output = fol.replace('/', '-') + '.txt'
        # Is there a better pattern to use?
        txt_files = glob.glob(os.path.join(dirname, fol, '*.txt'))
        print(" %s: %i files" % (fol, len(txt_files)))
        with smart_open(os.path.join(dirname, output), "wb") as n:
            for i, txt in enumerate(txt_files):
                with smart_open(txt, "rb") as t:
                    one_text = t.read().decode("utf-8")
                    for c in control_chars:
                        one_text = one_text.replace(c, ' ')
                    one_text = normalize_text(one_text)
                    all_lines.append(one_text)
                    n.write(one_text.encode("utf-8"))
                    n.write(newline)

    # Save to disk for instant re-use on any future runs
    with smart_open(os.path.join(dirname, 'alldata-id.txt'), 'wb') as f:
        for idx, line in enumerate(all_lines):
            num_line = u"_*{0} {1}\n".format(idx, line)
            f.write(num_line.encode("utf-8"))

assert os.path.isfile("data/aclImdb/alldata-id.txt"), "alldata-id.txt unavailable"
print("Success, alldata-id.txt is available for next steps.")

Cleaning up dataset...
 train/pos: 12500 files
 train/neg: 12500 files
 test/pos: 12500 files
 test/neg: 12500 files
Success, alldata-id.txt is available for next steps.


In [4]:
#define all of the functions
punctuation_list = list(string.punctuation)

def sent_tokenize(doc):
    sent_text = nltk.sent_tokenize(doc) # this gives you a list of sentences
    return sent_text

def word_tokenize(sent):
    tokenized_text = nltk.word_tokenize(sent)  # this gives you a list of words
    tokenized_text = [token.lower() for token in tokenized_text if token not in punctuation_list]  # optional: convert all words to lower case
    return tokenized_text

def readfile(filename):
    with open(filename,'r',encoding='UTF-8') as f:
        content = f.readlines()
    content = [x.strip() for x in content] 
    #strip()读出有效文件，形成一个list
    #split()读成有效文件，根据一行来形成一个list
    return content

#padding the sentence
#sentences是一个影评，就是一个train_data_word[0]
#max_words是影评中句子的最大含词量
#max_sents是影评中最大的句子个数
#保证每个影评的句子个数和句子长度都一样
def pad_sent(sentences, max_words, max_sents):
    """
    Pads sequences to the same length.
    Input: sentences - List of lists, where each element is a sequence.
    - max_words: Int, maximum length of all sequences.
    """
    # pad sentences in a doc
    sents_padded = pad_sequences(sentences, maxlen=max_words, padding='post') 
    # pad a doc to have equal number of sentences
    if len(sents_padded) < max_sents:
        doc_padding = np.zeros((max_sents-len(sents_padded),max_words), dtype = int)
        sents_padded = np.append(doc_padding, sents_padded, axis=0)
    else:
        sents_padded = sents_padded[:max_sents]
    return sents_padded

#build from word to integer as the input of ''
def build_vocab(corpus):
    """
    Builds a vocabulary mapping from word to index based on the corpus.
    Input: list of all samples in the training data
    Return: OrderedDict - vocabulary mapping from word to integer.
    """
    # Build vocabulary
    corpus_2d = []  # convert 3d corpus to 2d list
    for doc in corpus:
        for sent in doc:
            corpus_2d.append(sent)
    word_counts = Counter(itertools.chain(*corpus_2d))
    # Mapping from index to word (type: list)
    vocabulary = ['<PAD/>', '<UKN/>']   # 0 for padding, 1 for unknown words
    vocabulary = vocabulary + [x[0] for x in word_counts.most_common()]
    # Mapping from word to index
    #如何避免呢
    vocab2int = OrderedDict({x: i for i, x in enumerate(vocabulary)})
    return vocab2int

#****这个corpus是几维呢
def build_input_data(corpus, vocab2int, max_words, max_sents):
    """
    Maps words in the corpus to integers based on a vocabulary.
    Also pad the sentences and documents into fixed shape
    Input: corpus - list of samples, each sample is a list of sentences, each sentence is a list of words
    """
    corpus_int = [[[getWordIdx(word, vocab2int) for word in sentence]for sentence in sample] for sample in corpus]
    corpus_padded = []
    for doc in corpus_int:
        corpus_padded.append(pad_sent(doc, max_words, max_sents))
    corpus_padded = np.array(corpus_padded)    
    return corpus_padded

def load_embedding_matrix_gensim(embed_path, vocab2int, EMBEDDING_DIM):
    """
    load Word2Vec using gensim: 300x1 word vecs from Google (Mikolov) word2vec: GoogleNews-vectors-negative300.bin
    return embedding_matrix 
    embedding_matrix[i] is the embedding for 'vocab2int' integer index i
    """
    word2vec_model = gensim.models.KeyedVectors.load_word2vec_format(embed_path, binary=True)
    embeddings = {}
    embeddings['<PAD/>'] = np.zeros(EMBEDDING_DIM) # Zero vector for '<PAD/>' word
    embedding_UKN = np.random.uniform(-0.10, 0.10, EMBEDDING_DIM)  # Vector of small random numbers for unknown words
    # embedding_UKN = vector / np.linalg.norm(embedding_UKN)   # Normalize to unit vector
    embeddings['<UKN/>'] = embedding_UKN

    for word in word2vec_model.vocab:
        embeddings[word] = word2vec_model[word]

    embedding_matrix = np.zeros((len(vocab2int) , EMBEDDING_DIM))
    for word, i in vocab2int.items():
        embedding_vector = embeddings.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
        else:   # word is unknown
            embedding_vector = np.random.uniform(-0.10, 0.10, EMBEDDING_DIM)  # Vector of small random numbers for unknown words
            # embedding_vector = vector / np.linalg.norm(embedding_vector)   # Normalize to unit vector
            embedding_matrix[i] = embedding_vector

    return embedding_matrix


In [5]:
#**********loading data
#get the movie review "list of string"
test_neg = readfile('data/aclImdb/test-neg.txt')
test_pos = readfile('data/aclImdb/test-pos.txt')
train_neg = readfile('data/aclImdb/train-neg.txt')
train_pos = readfile('data/aclImdb/train-pos.txt')

#use these lists to label the movie reviews
test_neg_label = [0 for i in range(len(test_neg))]
test_pos_label = [1 for i in range(len(test_pos))]
train_neg_label =[0 for i in range(len(train_neg))]
train_pos_label =[1 for i in range(len(train_pos))]


#merge the test label
test_label = test_neg_label + test_pos_label

#merge the train label
train_label = train_neg_label + train_pos_label

#merge the test data
test_data = test_neg + test_pos

#merge the train data
train_data = train_neg + train_pos

#shuffule the these lists
from sklearn.utils import shuffle 
train_data , train_label = shuffle(train_data , train_label , random_state = 0)
test_data , test_label = shuffle(test_data ,test_label , random_state = 0) 

In [6]:
#**********tokenize the two lists of reviews into two lists of list of sentences
train_data_sent = [sent_tokenize(train_data[i]) for i in range(len(train_data))]
test_data_sent = [sent_tokenize(test_data[i]) for i in range(len(test_data))]  

#**********for training data prepocessing
#tokenize the two lists of list of sentences into two lists of list of list of word
train_data_word = [[]for i in range(len(train_data_sent))]
for i in range(len(train_data_sent)):
    for j in range(len(train_data_sent[i])):
        #some mistakes,I need to find a better to add element to the list
        word_tokens = word_tokenize(train_data_sent[i][j])
        if word_tokens != []:
            train_data_word[i].append(word_tokens)  

In [7]:
t_idx = 9
print(train_data[t_idx])
print(train_data_word[t_idx])

After ,  I watched the films .  .  .  I thought ,   " Why the heck was this film such a high success in the Korean Box Office ?  "  Even thought the movie had a clever / unusal scenario ,  the acting wasn't that good and the characters weren't very interesting .  For a Korean movie .  .  .  I liked the fighting scenes .  If you want to watch a film without thinking ,  this is the film for you .  But I got to admit .  .  .  the film was kind of childish .  .  .  6 / 10
[['after', 'i', 'watched', 'the', 'films'], ['i', 'thought', '``', 'why', 'the', 'heck', 'was', 'this', 'film', 'such', 'a', 'high', 'success', 'in', 'the', 'korean', 'box', 'office', '``'], ['even', 'thought', 'the', 'movie', 'had', 'a', 'clever', 'unusal', 'scenario', 'the', 'acting', 'was', "n't", 'that', 'good', 'and', 'the', 'characters', 'were', "n't", 'very', 'interesting'], ['for', 'a', 'korean', 'movie'], ['i', 'liked', 'the', 'fighting', 'scenes'], ['if', 'you', 'want', 'to', 'watch', 'a', 'film', 'without', 'th

In [8]:
#building the vacabulary
vocab_to_int_train = build_vocab(train_data_word)

#get the padding element
maxlen_word1 = 0
maxlen_sent1 = 0
#pad_train_set = []
list_maxlen_sent1 = []
list_maxlen_word1 = []
#get the list which is the maxim quantity of sentence
for i in range(len(train_data_sent)):
    list_maxlen_sent1.append((len(train_data_sent[i])))
#get the list which is the maxim quantity of word
for i in range(len(train_data_sent)):
    for j in range(len(train_data_sent[i])):
        list_maxlen_word1.append(len(train_data_sent[i][j]))
#get the max sentence
list_maxlen_sent1 = sorted(list_maxlen_sent1)
maxlen_sent1 = list_maxlen_sent1[int(len(list_maxlen_sent1)*0.95)]
#get the max words
list_maxlen_word1 = sorted(list_maxlen_word1)
maxlen_word1 = list_maxlen_word1[int(len(list_maxlen_word1)*0.95)]


#start to pad
train_copus_padded = build_input_data(corpus=train_data_word,max_sents=maxlen_sent1,max_words=maxlen_word1,vocab2int=vocab_to_int_train)


In [9]:
# **********for testing data prepocessing
# tokenize the two lists of list of sentences into two lists of list of list of word
test_data_word = [[]for i in range(len(test_data_sent))]
for i in range(len(test_data_sent)):
    for j in range(len(test_data_sent[i])):
        #some mistakes,I need to find a better to add element to the list
        test_data_word[i].append(word_tokenize(test_data_sent[i][j]))  
        
# #building the vacabulary
# vocab_to_int_test = build_vocab(test_data_word)

#start to pad test data
#25000*36*224
test_copus_padded = build_input_data(corpus=test_data_word,max_sents=maxlen_sent1,max_words=maxlen_word1,vocab2int=vocab_to_int_train)

In [10]:
print(train_copus_padded.shape)
print(test_copus_padded.shape)

print(train_data_sent[8], '\n')
print(train_data_word[8], '\n')
print(train_copus_padded[8])

(25000, 36, 224)
(25000, 36, 224)
['This is a low budget Roger Corman horror / creature flick .', 'A DinoCroc is created when manipulation of prehistoric genes runs amok .', 'An engineered croc first kills one of its own then gets the taste of human and becomes a fast growing terror after escaping .', 'None of the characters have any depth ,  but then they are not the focal point .', 'We only get a few glimpses of the huge two-legged dinosaur descendant and some of the best  " kill "  scenes in a small budget film .', 'My favorite scene is of a moronic character trying to use a three legged dog for bait and becomes croc food himself .', 'Nothing left on the pier but ankle top feet .', 'With no real stand out roles :  Jane Longendecker ,  Bruce Weitz and Charles Napier .', 'Most pathetic is Matt Borlenghi and an obnoxious professional croc hunter Costas Mandylor .', 'I was most impressed with the alluring Joanna Pacula as the respectfully feared Dr .', 'P .', 'DINOCROC is redeeming as a

In [11]:
# use the pickle to store the data
file = open('pickle_data/train_label.pickle','wb')
pickle.dump(train_label,file)
file.close()

file = open('pickle_data/test_label.pickle','wb')
pickle.dump(test_label,file)
file.close()

file = open('pickle_data/train_data_word.pickle','wb')
pickle.dump(train_data_word,file)
file.close()

file = open('pickle_data/test_data_word.pickle','wb')
pickle.dump(test_data_word,file)
file.close()

file = open('pickle_data/train_data_sent.pickle','wb')
pickle.dump(train_data_sent,file)
file.close()

file = open('pickle_data/test_data_sent.pickle','wb')
pickle.dump(test_data_sent,file)
file.close()

file = open('pickle_data/vocab_train.pickle','wb')
pickle.dump(vocab_to_int_train,file)
file.close()

file = open('pickle_data/train_copus_pad.pickle','wb')
pickle.dump(train_copus_padded,file)
file.close()

file = open('pickle_data/test_copus_pad.pickle','wb')
pickle.dump(test_copus_padded,file)
file.close()

In [12]:
# get the embedding matrix
#the number of the vocabulary is 100327
#把每个词映射到一个300维度的vector
#这个matrix是二维的
#用vocab2int中每个词对应的整数来去matrix来找对应的vector
dimension = 300
path = '~/Desktop/NLP_resources/GoogleNews-vectors-negative300.bin'
embedding_matrix = load_embedding_matrix_gensim(embed_path = path,vocab2int=vocab_to_int_train,EMBEDDING_DIM=dimension)

#use pickle to store the data
file = open('pickle_data/embedding_matrix','wb')
pickle.dump(embedding_matrix,file)
file.close()

In [13]:
print(len(vocab_to_int_train))
embedding_matrix.shape

97162


(97162, 300)